# Model-free Methods

In [ ]:
using Match, Plots
using Distributions, StatsBase, RollingFunctions, Random
include("operators.jl")

# Definition of the environment (MDP)

State:

In [ ]:
S = [if i==j && i==2 () else (i,j) end for i=1:3,j=1:4 ]

For convenience can also be expressed as a unidimensional array:

In [ ]:
Sₗ = reshape(S, 12,1)
Sₗ = [ s for s in Sₗ if s != ()]


Actions

In [ ]:
A = ["↑","↓","←","→"]

Function for checking that actions are valid within our environment:

In [ ]:
OK(xy) = if xy[1] > 0 && xy[2] > 0 && xy[1] <= size(S,1) && xy[2] <= size(S,2) && S[xy[1],xy[2]] != ()  true else false end


We also need to know which states are terminal:

In [ ]:
is_terminal(s) = if s == (1,4) || s == (2,4) true else false end 

Function returning the next state if we take a given action:

In [ ]:
Af((i,j),a) =  @match a begin
                
    "↑" =>  if OK((i-1,j)) 
                (i-1,j) 
            else (i,j) end
    "↓" =>  if OK((i+1,j))
                (i+1,j) 
            else (i,j) end
    "←" => if OK((i,j-1)) 
                (i,j-1) 
            else (i,j)  end
    "→" => if OK((i,j+1))
            (i,j+1)
        else (i,j)  end
    end

Note as this is a model-free method **the agent does not know the transition probabilities**.

Reward function

In [ ]:
R = [@match (i,j) begin (2,2)=> nothing
                        (1,4) => 1. 
                        (2,4) => -1. 
                        _ => 0.
                    end for i=1:3,j=1:4 ]

The reward for a particular state:

In [ ]:
r(s) = R[s[1],last(s)]

The discount factor ($\gamma$)

In [ ]:
γ = 0.9

Γ(s) = @match s begin
            (1,4) => 0.0 #Terminal state
            (2,4) => 0.0 #Terminal state
            _ => γ
        end


# Q-learning algorithm

## Q-values initialisation

Initialise arbitrary to 0 for every action:

In [ ]:
Q₀() = begin
    Q₀v = repeat([("",0.)],3,4,4) # size of State space and Action space
    [ Q₀v[i,j,k] = (A[k],0.0) for k=1:4,j=1:4 ,i=1:3  ]
    return Q₀v
end

## Create a policy to use in the learning process

### We will use a $\epsilon$*-greedy* policy

* With a probability $\epsilon$ the agent takes any action (random)
* With a probability $( 1 - \epsilon)$ the agent takes the action with max $Q$*-value*


We need to define the probabilities of each action in our policy:

In [ ]:
Pϵ(Q,s) = begin
    x,y = s
    Aₛ = Q[x,y,:]
    Aₛ = sort(Aₛ,by=last,rev=true) #sort estimated Q value-action
    
    #get the index of the action with max Q-value
    a⁺ᵢ = first(indexin([first(first(Aₛ))],A))

    # Fill out probabilities according to ϵ-greedy
    P = repeat([ϵ/(length(A) - 1)],length(A))
    P[a⁺ᵢ] = 1 - ϵ

    return P
end

Define $\epsilon$ and $\alpha$ parameters

In [ ]:
ϵ = 0.05
α = 0.8

Now we define the policy that uses the defined probabilities:

In [ ]:
πϵ(Q,s) = begin
    Pₛ = Pϵ(Q,s)
    a = sample(A,Weights(Pₛ))
end

### main algorithm

In [ ]:
q_learning(n) = begin

    # Initialise Q arbitrary   
    Qᵥ = Q₀()
    
    #reset our environment 
    s₀ = (3,1) 
    #s₀ = sample(Sₗ) 


    # For storing stats
    lgs = [] # length of episodes
    rtns = [] # return of episodes

    # For n episodes

    for episode = 1:n
        # restart the agent in initial position
        s = s₀ 
        
        #Accumulator of rewards for the episode
        rtn = 0 #episode return
        
        # Arbitrary number of steps to try
        for step = 1:100
            
            x,y = s
            
            # Get the action to take from our ϵ-greedy policy
            a = πϵ(Qᵥ,s)
            # index of the action
            aᵢ = first(indexin([a],A))

            #For current state calculates:
            
            # next state
            s′ = Af(s,a)
            x′,y′ = s′

            # reward
            rₛ = r(s)
            # accumulate episode return
            rtn = rtn + rₛ
            
            # maxₐ Q(S′,A)
            A′ = Qᵥ[x′,y′,:]
            a′ = first(sort(A′,by=last,rev=true))           
            a′ᵢ = first(indexin([first(a′)],A)) 

            # Temporal difference target: R + γ maxₐ Q(S′,A)
            td⁺ = rₛ + Γ(s) * last(Qᵥ[x′,y′,a′ᵢ])
            
            # Temporal difference: R + γ maxₐ Q(S′,a) - Q(S,A)
            tdΔ = td⁺ - last(Qᵥ[x,y,aᵢ])

            # Q(S,A) = Q(S,A) + α [R + γ maxₐ Q(S′,a) - Q(S,A)]
            newQvalue = last(Qᵥ[x,y,aᵢ]) + α * tdΔ

            # update Qᵥ array
            newQᵥ = copy(Qᵥ)
            newQᵥ[x,y,aᵢ] = ("$(A[aᵢ])", newQvalue)
            Qᵥ = newQᵥ

            #if in terminal state stores stats and break steps loop
            if is_terminal(s)
                push!(lgs,step)
                push!(rtns,rtn)
                break
            end
            # takes step 
            s = s′
        end
    end
   return Qᵥ,lgs,rtns
end

In [ ]:
Qᵥ,lgts,rtns = q_learning(1000)

Convergence:

The resulting policy is:

In [ ]:
[ 
    @match (i,j) begin
        (2,2)   => ()
        (1,4) => ""
        (2,4) => ""
        _ => first(first(sort(Qᵥ[i,j,:],by=last,rev=true)))
    end
    for i=1:3,j=1:4 
]

Calculate reward convergence with a moving average (window size = 10 ep.)

In [ ]:

lgts_rm = rollmean(lgts * 1.,10)
rtns_rm = rollmean(rtns * 1.,10);

In [ ]:
plot(plot(rtns_rm ./ lgts_rm),xlabel="episode",ylabel="mean (Reward / Step)")

Number of steps required in each episode:

In [ ]:
plot(lgts_rm,xlabel="episodes",ylabel="steps")